### This project is illustrating how to use neural style transfer for both images and videos:
> The necessary libaries and framework would be put in the requirement.txt but i'll still indicate them below:
* matplotlib.pyplot
* tensorflow
* tensorflow_hub
* numpy

In [1]:
# Importing the necessary libaries
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2

c:\Users\hp\anaconda3\envs\newenv001\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\hp\anaconda3\envs\newenv001\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\hp\anaconda3\envs\newenv001\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### *Style Transfer for images:*

In [ ]:
def preprocess_and_view_image(content_image_path, style_image_path, save_path=None):
    content_image = plt.imread(content_image_path)
    style_image = plt.imread(style_image_path)
    # Scale the images to ensure they are within the range of [0, 1]
    content_image = content_image.astype(np.float32)[np.newaxis, ...] / 255.
    style_image = style_image.astype(np.float32)[np.newaxis, ...] / 255.

    # Resize the style image
    style_image = tf.image.resize(style_image, [256, 256])
    # Load the module
    hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
    outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
    stylized_image = outputs[0]
    # Convert the tensor to a NumPy array
    stylized_image_array = stylized_image.numpy()

    # Ensure values are within the valid range [0, 1]
    stylized_image_array = np.clip(stylized_image_array, 0, 1)
    # Visualize the outputs
    plt.figure(figsize=(15, 5))
    plt.subplot(131)
    plt.imshow(content_image[0])
    plt.title('Content Image')
    plt.axis(False)
    plt.subplot(132)
    plt.imshow(style_image[0])
    plt.title('Style Image')
    plt.axis(False)
    plt.subplot(133)
    plt.imshow(stylized_image_array[0])
    plt.title('Stylized Image')
    plt.axis(False)
    plt.show()

    # Save the stylized image if a save path is provided
    if save_path is not None:
        plt.imsave(save_path, stylized_image_array[0])

In [ ]:
preprocess_and_view_image('/content/drive/MyDrive/Neural Transfer Project/Content/spider-man-across-5120x2880-10140.jpg',
                        '/content/drive/MyDrive/Neural Transfer Project/Style/frida.jpg',
                        '/content/drive/MyDrive/Neural Transfer Project/Generated Output/stylized_image.jpg')


*The purpose of adding the `np.new_axis` is to enable the module, which has been trained to process data in batches, to effectively handle individual data samples as if they were part of a batch. This adjustment ensures compatibility between the module's expectations and the input data, allowing seamless processing and accurate results.*

*Adjusting the size of the style image is necessary because the module was originally trained on style images with dimensions of (256,256) pixels.*

*I adjusted the dimensions of the `style` image to 256 by 256 pixels, aligning with the size used during training of the style transfer network. Meanwhile, the `content` image size can be customized according to your preferences.*

### *Style Transfer for imported Videos from user device:*

In [ ]:
def preprocess_and_display_video(video_path, style_image_path):
    '''This function preprocesses the video and applies style transfer'''

    # Load our style transfer module
    hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

    # Capture the video file and transform it into a VideoCapture object
    cap = cv2.VideoCapture(video_path)

    # Preprocess the style image
    style_image = plt.imread(style_image_path)
    style_image = style_image.astype(np.float32)[np.newaxis, ...] / 255.
    style_image = tf.image.resize(style_image, [256, 256])

    stylized_frames = []  # List to store stylized frames

    # Create a while loop to read and stylize each frame of the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # preprocess the video
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = frame.astype(np.float32)[np.newaxis, ...] / 255
        frame = tf.image.resize(frame, [256, 256])

        # Apply stylization
        stylized_output = hub_module(tf.constant(frame), tf.constant(style_image, dtype=tf.float32))
        stylized_frame = (stylized_output[0].numpy()[0] * 255).astype(np.uint8)
        stylized_frame = cv2.cvtColor(stylized_frame, cv2.COLOR_RGB2BGR)

        stylized_frames.append(stylized_frame)

    # Release video object
    cap.release()

    return stylized_frames

In [ ]:
stylized_frames = preprocess_and_display_video("temp_video.mp4", 'Style\monet.jpeg')
    
# Create a video from stylized frames using OpenCV's VideoWriter
frame_height, frame_width, _ = stylized_frames[0].shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("stylized_video.mp4", fourcc, 30, (frame_width, frame_height))

for frame in stylized_frames:
    out.write(frame)
out.release()

# Read the stylized video back and display it using Streamlit
stylized_video = open("stylized_video.mp4", "rb").read()

### *Style Transfer for real time videos:*

In [2]:
def real_time_transfer(style_image_path):
    ''' This function helps in stylizing videos directly from our camera'''
    hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
    org_video = cv2.VideoCapture(1)  # This captures the original/real-time videos from the webcam
    for webcam_index in [0, 1]:
        org_video = cv2.VideoCapture(webcam_index)
        if org_video.isOpened():
            break
            
    if not org_video.isOpened():
        print('Error reading video file')

    # Get the frame_height and width to mirror the actual video we want to stylize
    frame_width = int(org_video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(org_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(org_video.get(cv2.CAP_PROP_FPS))

    # Preprocess the style image
    style_image = plt.imread(style_image_path)
    style_image = style_image.astype(np.float32)[np.newaxis, ...] / 255.
    style_image = tf.image.resize(style_image, [256, 256])

    # Create VideoWriter object
    out = cv2.VideoWriter('stylized_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    while org_video.isOpened():
      ret, frame = org_video.read()
      if not ret:
        break

    # preprocess the video
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      frame = frame.astype(np.float32)[np.newaxis, ...] / 255
      frame = tf.image.resize(frame, [256, 256])

    # Apply stylization
      stylized_output = hub_module(tf.constant(frame), tf.constant(style_image, dtype=tf.float32))
      stylized_video_output = stylized_output[0].numpy()[0]

    # postprocess the video
      stylized_video_output = (stylized_video_output * 255).astype(np.uint8)
      stylized_video_output = cv2.cvtColor(stylized_video_output, cv2.COLOR_RGB2BGR)

      out.write(stylized_video_output)
    
    # Display the stylized video in real-time
      cv2.imshow('Stylized Video', stylized_video_output)
    
    # Check if the user pressed the 'q' key to quit
      if cv2.waitKey(1) & 0xFF == ord('q'):
         break
    org_video.release()
    out.release()
    cv2.destroyAllWindows()
    print('Video successfully stylized')


In [5]:
real_time_transfer("Style\monet.jpeg")

Video successfully stylized
